In [1]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy
import matplotlib.colors as mcolors
import matplotlib.cm as cm

import os
import random
from gensim.models import Word2Vec
import networkx as nx

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
node_df = pd.read_csv("2024_05_05_meta_node_data.csv")
edge_df = pd.read_csv("2024_04_24_meta_edge_sourcetarget_disease_pub_pli_reciprocal.csv")
geneGraph = nx.DiGraph()
%run vizfunctions.ipynb
node_df.set_index('id',inplace=True)
node_df.head()

/var/folders/6q/0qnxsny943385fwzz738241c0000gn/T/ipykernel_58921/4103866634.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  edge_df = pd.read_csv("2024_04_24_meta_edge_sourcetarget_disease_pub_pli_reciprocal.csv")


,symbol,disease_assoc_cat,publication_count,ortholog_count,all_ortho_count
id,,,,,
7414,VCL,disease_assoc,279,5,"{'hsapien': {'entrezid': 7414, 'symbol': 'VCL'..."
4626,MYH8,disease_assoc,36,4,"{'hsapien': {'entrezid': 4626, 'symbol': 'MYH8..."
9722,NOS1AP,disease_assoc,141,5,"{'hsapien': {'entrezid': 9722, 'symbol': 'NOS1..."
9891,NUAK1,non_disease_assoc,70,5,"{'hsapien': {'entrezid': 9891, 'symbol': 'NUAK..."
81788,NUAK2,disease_assoc,46,2,"{'hsapien': {'entrezid': 81788, 'symbol': 'NUA..."


In [3]:
# Removing all non_omim genes for training/testing
edge_df = edge_df[
    (edge_df['source_disease_assoc_cat'] != 'non_omim') &
    (edge_df['target_disease_assoc_cat'] != 'non_omim')
]

In [35]:
# Generating vectors
create_graph(10000)   # 1048576
walks = generate_random_walks(geneGraph, 50, 10, 0.5, 0.5)
str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, vector_size=128, window=5, min_count=0, sg=0, workers=2, hs=0, epochs=1)
node_ids = list(model.wv.index_to_key)  # node ids
int_node_ids = []
for node_id in node_ids:
    int_node_ids.append(int(node_id))
int_node_ids = numpy.array(int_node_ids)
vectors = (model.wv.vectors) # vectors

%store vectors

# 30000 edges -- 20 seconds (~3800 nodes)
# 100000 -- 11 minutes (~9400 nodes)

Stored 'vectors' (ndarray)


/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/env/lib/python3.12/site-packages/IPython/extensions/storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/vectors requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


In [10]:
%store int_node_ids
%store vectors


Stored 'int_node_ids' (ndarray)
Stored 'vectors' (ndarray)


/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/env/lib/python3.12/site-packages/IPython/extensions/storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/vectors requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


In [37]:
# Creating a dataset with just the genes, classification, and vectors
vector_data = pd.DataFrame(vectors)
vector_data['gene'] = int_node_ids

if node_df.index.name != 'id':
    node_df.set_index('id', inplace=True)

disease_assoc_cat = numpy.array([node_df.loc[node_id]['disease_assoc_cat'] for node_id in int_node_ids])
vector_data['disease_assoc_cat'] = disease_assoc_cat

# vector_data.set_index('gene',inplace=True)

# Creating features
X = vector_data.iloc[:, :-1].values
y = vector_data['disease_assoc_cat'].values

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initializing the classifier
mlp_model = MLPClassifier(activation='tanh', alpha=0.0001, hidden_layer_sizes=(100, 100, 50), learning_rate='adaptive', max_iter=400, solver='adam') # parameters found using GridSearchCV

# Train the classifier
mlp_model.fit(X_train_scaled, y_train)

# # Evaluate the model
# y_pred = mlp_model.predict(X_test_scaled)
# report = classification_report(y_test, y_pred, output_dict=True)

# Evaluate the classifier
accuracy = mlp_model.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

# 30000 edges -- 5 seconds (~3800 nodes) --> accuracy was around 0.621
# 100000 -- 2 minutes (~9400 nodes) --> accuracy was around 0.051

#    vector size       edges    nodes       random walk               accuracy               
#         128         10000     ~3800     (50, 10, 0.5, 0.5)           0.621
#         256         10000     ~3800     (50, 10, 0.5, 0.5)       0.5338345864661654
#         512         10000     ~3800     (50, 10, 0.5, 0.5)       0.553030303030303
#         64          10000     ~3800     (50, 10, 0.5, 0.5)       0.014814814814814815

#         64          10000     ~3800     (50, 10, 0.8, 0.4)       0.6330935251798561
#         128         10000     ~3800     (50, 10, 0.8, 0.4)       0.635036496350365
#         256         10000     ~3800     (50, 10, 0.8, 0.4)       0.6343283582089553
#         512         10000     ~3800     (50, 10, 0.8, 0.4)       0.6370370370370371

#         64          10000     ~3800     (50, 10, 0.4, 0.8)       0.5378787878787878
#         128         10000     ~3800     (50, 10, 0.4, 0.8)       0.5793650793650794
#         256         10000     ~3800     (50, 10, 0.4, 0.8)       0.6377952755905512
#         512         10000     ~3800     (50, 10, 0.4, 0.8)       0.6060606060606061


Accuracy: 0.6071428571428571


In [10]:
# Range of parameters to iterate over
param_combinations = [
    (num_walks, walk_length, p, q)
    for num_walks in [100, 300, 500]
    for walk_length in [10, 50, 100, 500]
    for p in [0.25, 0.5, 1, 2]
    for q in [0.25, 0.5, 1, 2]
]

# Initialize results dictionary
results = {}

# Iterate over the parameter combinations
for num_walks, walk_length, p, q in param_combinations:
    # Generate vectors
    vectors, int_node_ids = generate_vectors(geneGraph, num_walks, walk_length, p, q)
    
    # Evaluate the model
    report = train_model(vectors, int_node_ids)#vectors, int_node_ids, node_df)
    
    # Store the results
    param_key = f"num_walks={num_walks}_walk_length={walk_length}_p={p}_q={q}"
    results[param_key] = report

# Print the results
for key, value in results.items():
    print(f"Parameters: {key}")
    print(f"Report: {value}")

/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 

Parameters: num_walks=100_walk_length=10_p=0.25_q=0.25
Report: {'disease_assoc': {'precision': 0.5, 'recall': 0.5, 'f1-score': 0.5, 'support': 2.0}, 'non_disease_assoc': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0}, 'accuracy': 0.3333333333333333, 'macro avg': {'precision': 0.25, 'recall': 0.25, 'f1-score': 0.25, 'support': 3.0}, 'weighted avg': {'precision': 0.3333333333333333, 'recall': 0.3333333333333333, 'f1-score': 0.3333333333333333, 'support': 3.0}}
Parameters: num_walks=100_walk_length=10_p=0.25_q=0.5
Report: {'disease_assoc': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1.0}, 'non_disease_assoc': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 2.0}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3.0}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3.0}}
Parameters: num_walks=100_walk_length=10_p=0.25_q=1
Report: {'disease_assoc': {'precision': 0.6666

/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with 

In [ ]:
# Using gridsearchcv for hyperparameter training
param_grid = {
    'hidden_layer_sizes': [(100, 100, 50), (150, 150, 150), (150, 100, 100), (100, 100, 150), (150, 100, 50)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [200, 300, 400]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(mlp_model, param_grid, cv=3, verbose=2, n_jobs=-1)

# Perform the grid search on the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found by the grid search
print("Best parameters found: ", grid_search.best_params_)